# Tarefa 01

---

**1.** Monte um passo a passo para o Bagging

O método Bagging (Bootstrap Aggregating) opera conforme os seguintes procedimentos:

1. **Bootstrap:** O processo de bootstrap inicia com a geração de amostras aleatórias de treinamento com reposição a partir do conjunto original de dados de treinamento. Cada amostra bootstrap mantém o mesmo tamanho do conjunto de dados original, permitindo que cada exemplo seja selecionado várias vezes.

2. **Modelagem (Base learners):** Para cada amostra bootstrap criada, um modelo de Machine Learning é treinado de maneira independente. Isso resulta na criação de múltiplos modelos, sendo que cada um é treinado em uma amostra bootstrap distinta. Esses modelos são referidos como base learners.

3. **Agregação (Aggregating):** Nesta fase, os resultados dos modelos individuais são agregados para produzir uma única previsão final. Em problemas de classificação, a agregação frequentemente é realizada por votação majoritária, na qual a classe mais comum entre os modelos é escolhida como a previsão final. Para problemas de regressão, a agregação é efetuada calculando-se a média dos resultados dos modelos.

**2.** Explique com suas palavras o Bagging

O método ``Bagging``, ou ``Bootstrap Aggregating``, é uma técnica de combinação de modelos amplamente utilizada em Machine Learning. Este método consiste na geração de múltiplas amostras de treinamento, em que cada amostra é criada aleatoriamente com reposição a partir do conjunto de dados original, em um procedimento conhecido como Bootstrap. Posteriormente, cada amostra é utilizada para treinar um modelo independente, denominado base learner.

Durante a fase de teste, os modelos são aplicados aos dados de teste e suas previsões são combinadas por meio de uma estratégia de agregação. Em problemas de classificação, a combinação geralmente é realizada através de votação majoritária, onde a classe mais frequente entre os modelos é selecionada como a classe final. Já em problemas de regressão, a combinação é efetuada calculando-se a média das previsões dos modelos.

**3.** (Opcional) Implementar em python o código do Bagging
   - Bootstrap
   - Modelagem
   - Agregação

In [1]:
# Import das bibliotecas

import pandas as pd

from sklearn.datasets        import load_iris
from sklearn.datasets        import load_diabetes

from sklearn.model_selection import train_test_split

from sklearn.tree            import DecisionTreeClassifier
from sklearn.metrics         import accuracy_score

from sklearn.tree            import DecisionTreeRegressor
from sklearn.metrics         import mean_squared_error, r2_score

In [2]:
# Exemplo da técnica de bagging para problemas de classificação

X = load_iris().data
y = load_iris().target

df = pd.DataFrame(X, columns=load_iris().feature_names)
df['target'] = y

def bagging_classifier(df:pd.DataFrame, 
                       num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                       test_size:float=0.25
                      ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        y_train = df_train['target']
        
        # Modelagem (base learners)
        model = DecisionTreeClassifier()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test)})
    
    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mode(axis=1)  # Agregando o valor com maior número de aparições nas predições dos modelos
                .rename(columns={0:'y_pred'}))
 
    # Resultados
    print(model)
    print('Accuracy score:', accuracy_score(y_true=y_test, 
                                            y_pred=y_pred['y_pred']
                                           ))

    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred['y_pred'].astype(int)], 
                     axis=1)

bagging_classifier(num_bootstrap_samples=10, df=df, test_size=0.33)

DecisionTreeClassifier()
Accuracy score: 0.9


,y_test,y_pred
0,2,2
1,2,2
2,0,0
3,0,0
4,0,0
5,1,1
6,2,2
7,2,2
8,1,1
9,2,2


In [3]:
# Exemplo da técnica de bagging para problemas de regressão

X = load_diabetes().data
y = load_diabetes().target

df = pd.DataFrame(X, columns=load_diabetes().feature_names)
df['target'] = y

def bagging_regressor(df:pd.DataFrame, 
                      num_bootstrap_samples:int=3,  # Parâmetro da função que define a quantidade de amostragens para treinamento
                      test_size:float=0.25
                      ) -> pd.DataFrame:
    
    df_train, df_test = train_test_split(df, test_size=test_size)
    
    X_test = df_test.drop(['target'], axis=1)
    y_test = df_test['target'].rename('y_test')
    
    # Dicionário para os resultados das predições de cada modelo
    y_pred_bagging = {}

    for i in range(num_bootstrap_samples):
        # Bootstrap
        df_train = df_train.sample(n=len(df_train), 
                                   replace=True)  # Amostragem COM reposição

        X_train = df_train.drop(['target'], axis=1)
        y_train = df_train['target']

        # Modelagem (base learners)
        model = DecisionTreeRegressor()
        model.fit(X_train, y_train)
        
        # Adicionando os resultados do modelo ao dicionário para agregação das predições
        y_pred_bagging.update({i:model.predict(X_test)})

    # Aggregating
    y_pred = (pd.DataFrame(y_pred_bagging)
                .mean(axis=1)  # Agregando as predições dos modelos baseando n a média dos resultados
                .rename('y_pred'))
 
    # Resultados
    print(model)
    print('Mean squared error:', mean_squared_error(y_true=y_test, 
                                                   y_pred=y_pred))
    print('Coefficient of determination:', r2_score(y_true=y_test, 
                                                    y_pred=y_pred))
    
    return pd.concat(objs=[y_test.reset_index(drop=True), 
                           y_pred], 
                     axis=1)
    
bagging_regressor(num_bootstrap_samples=100, df=df, test_size=0.33)

DecisionTreeRegressor()
Mean squared error: 4563.843574657535
Coefficient of determination: 0.2088265010867576


,y_test,y_pred
0,131.0,111.22
1,64.0,94.75
2,257.0,113.69
3,308.0,247.69
4,233.0,223.66
...,...,...
141,120.0,136.02
142,173.0,167.76
143,65.0,95.24
144,111.0,115.75


---